# Devoir 3
Kevin Chalifoux (17 110 582)
/ Julien Corriveau-Trudel (17 090 489)
/ Gabriel Dupuis (17 022 516)

In [1]:
using LinearAlgebra;
import Pkg; Pkg.add("ForwardDiff");
using ForwardDiff;
IJulia.clear_output();

# Algorithmes

## Gradient conjugué pour cas quadratique
Algorithme du gradient conjugué appliqué au cas quadratique, où 

$$f : \mathbb{R}^n \rightarrow \mathbb{R}$$
$$x \mapsto x^tQx + c^tx$$


In [39]:
# GC_quad : Algorithme du gradient conjugué appliqué au cas quadratique x^t*Q
#
# Entrée: Q: Matrice définie positive de la forme quadratique
#         c: Vecteur colonne de la forme quadratique
#         x0: Point de départ de l'algorithme
# Sortie: Point considéré comme minimum local

function GC_quad(Q, c, x0)
    
    # Rend les vecteurs en ligne ou en colonne, selon le cas
    try
        size(c)[2]
    catch
        if(typeof(c) == Array{Float64,1} || typeof(c) == Adjoint(Any) || typeof(c) == Adjoint(Float64,Array{Float64,1}))
            c = c'
        end
    end

    if size(x0)[1] == 1
        x0 = x0'
    end
    
    # Vérifie les dimensions des arguments
    if size(Q)[1] != size(Q)[2]
        return ArgumentError("Argument Q non carré.")
    elseif size(Q)[1] != size(c)[2]
        return ArgumentError("Argument c de dimension incompatible avec celles de Q.")
    elseif size(Q)[1] != size(x0)[1]
        return ArgumentError("Argument x0 de dimension incompatible avec celles de Q.")
    end

    dim = size(Q)[1]
    
    # Initialisation
    β = 0.0
    d = [0 for i in 1:dim]
    xk = x0
    gk = (xk'*Q+c)
    for k in 1:dim
        d = -gk' + β*d
        θ = -(gk*d)[1]/(d'*Q*d)[1]
        xk = xk + θ*d
        gk = (xk'*Q+c)
        if norm(gk) < eps()
            #print("SOLUTION = ",xk, "\n")
            print("...Fin à itération ",k, "\n")
            return xk
        end
        
        β = (gk*Q*d)[1]/(d'*Q*d)[1]
    end
    print("...Fin à itération ",dim, "\n")
    #print("SOLUTION = ",xk, "\n")
    return xk
end;

## Pas admissible
Cet algorithme sera nécessaire pour la version non-quadratique. On utilisera la mise en oeuvre d'un pas admissible tel que vu à la page 150 des NdC de J.-P. Dussault.

In [3]:
function Pas_Admissible(x, d, f::Function, τ, ∇f)
    θ = 1.0
    while f(x+θ*d)- f(x) > θ*τ*(∇f*d)[1]
        θ = θ/2
    end
    return θ
end;

## Gradient conjugué appliqué à des fonctions quelconques (non-quadratique)

In [43]:
@enum type_β FR PR HS HZ
# GC : Algorithme du gradient conjugué appliqué au cas quadratique x^t*Q
#
# Entrée: f (Function): fonction à minimiser
#         x0 (Array{Float64,1}): Point de départ de l'algorithme
#         τ (Float64 ∈[0,0.5]): paramètre de pas admissible pour le critère d'Armijo
#         variante_β (type_β): Variante du calcul de β_k
# Sortie: Point considéré comme minimum local de la fonction
function GConj(f::Function, x0, τ, variante_β, arrêt = Inf, ϵ::Float64 = sqrt(eps()))
    
    if size(x0)[1] == 1
        if size(x0)[2] != 1
            return ArgumentError("Argument x0 vecteur ligne. Doit être vecteur colonne.")
        end
    end
    dim = size(x0)[1]
    
    β = 0.0
    d = [0 for i in 1:dim]
    xk = x0
    #print("xk = ",xk, "\n")
    g = x -> ForwardDiff.gradient(f, x);
    
    gk_prec = g(xk)
    i = 0
    while norm(g(xk)) >= ϵ && i < arrêt
        for k in 1:dim
            #print("Itération ",k, "\n")
            gk = g(xk)
            #print("Norme gk=", norm(gk), "\n")
            if(k == 1)
                d = (-gk)
            else
                d = (-gk + β*d)
            end
            if (gk'*d)[1] >= 0
                i = i + 1
                #print("Broke out @", i, "\n")
                break
            end

            θ = Pas_Admissible(xk, 100*d, f, τ, gk')
            #print("Pas admissible: ", θ, "\n")
            
            gk_prec = gk
            
            xk = xk + 100*θ*d
            
            gk = g(xk)
            # Sortir de la boucle si on a gagné
            if norm(gk) < ϵ
                print("...Fin à itération ",i, "\n")
                return xk
            end
            
            if variante_β == FR #Fletcher-Reeves
                β = norm(gk)^2/norm(gk_prec)^2
            elseif variante_β == PR #Polak-Ribière
                #yk = (g(xk) - gk_prec)
                β = (norm(gk) - (gk'*gk_prec)[1])/norm(gk_prec)^2
            elseif variante_β == HS #Heslenes-Stiefet
                yk = (gk - gk_prec)
                ykd = (yk'*d)[1]
                #print("(Déno:", (yk'*d)[1], ") ")
                if ykd == 0.0
                    i = i + 1
                    #print("Broke out @", i, ", because deno is too small.\n")
                    break
                end
                β = (gk'*yk)[1]/(yk'*d)[1]
            elseif variante_β == HZ #Hager-Zhang
                yk = (gk - gk_prec)
                ykd = (yk'*d)[1]
                if ykd == 0.0
                    i = i + 1
                    break
                end
                β = ((yk'*gk)[1] - 2*(norm(yk)^2/ykd)*(d'*gk)[1])/ykd
            end
            i = i + 1
        end
    end
    print("...Fin à itération ",i, "\n")
    return xk
end;

# Tests

## Fonctions quadratiques de différentes dimensions ($f(x) = \frac{1}{2}x^TQx + bx)$, $x\in \mathbb{R}^n$)
On souhaite tester la fonction ``GC_quad()`` avec des fonctions quadratiques de dimension différentes. Dans cette optique, on choisit des dimensions extrêmes et moyennes, i.e. $n \in \{1, 5, 25, 1000\}$. Ce choix de dimensions testent:

* Est-ce que la fonction gère le cas trivial, où la dimension est 1? Cela revient à la recherche linéaire d'une fonction quadratique. 
* Est-ce que la fonction donne une réponse plausible lorsque les dimensions sont petites?
* Est-ce que la fonction donne une réponse plausible lorsque les dimensions sont grandes?

Pour générer ces fonctions quadratiques, on requiert des matrices symétriques définies positives, que nous générons par le processus inverse de la diagonalisation: On prend une matrice diagonale $D$ dont les éléments sont positives, donc que les valeurs propres sont positives, et on multiplie à gauche et à droite de $D$ par $P^{T}$ et $P$, où $P$ est inversible.

En effet, une telle matrice $Q = P^TDP$ est tel que $\forall y \in \mathbb{R}^n \backslash \{0\}$, $y^TQy = y^TP^TDPy = (Py)^TQPy > 0$, car $P$ inversible $\Rightarrow Py \neq 0$ et car $D$ est diagonale strictement positive. 

Les vecteurs $c$ seront générés aléatoirement, et n'impacteront pas la convexité des fonctions.

Le choix de point de départ est trivial, car on a une fonction quadratique, et donc qu'à partir de tous points de départ, l'algorithme arrivera théoriquement à la solution. Nous choisissons donc les $x_{\text{départ}}= [x_1 \ldots x_n]^T$ t.q. $x_i = 1, \forall i \in \{1, \ldots, n\}$.

Générons les matrices $Q$, les vecteurs $c$ et les points de départs.

In [5]:
D = Diagonal(100*rand(Float64,1000,1000))
P = UpperTriangular(rand(Float64,1000,1000)) + Diagonal(100*rand(Float64,1000,1000))
D1000 = D
Q1000 = P'*D*P

D = Diagonal(100*rand(Float64,25,25))
P = UpperTriangular(rand(Float64,25,25)) + Diagonal(25*rand(Float64,25,25))
D25 = D
Q25 = P'*D*P

D = Diagonal(100*rand(Float64,5,5))
P = UpperTriangular(rand(Float64,5,5)) + Diagonal(5*rand(Float64,5,5))

D5 = D
Q5 = P'*D*P

D = Diagonal(100*rand(Float64,1,1))
D1 = D
Q1 = D;

In [6]:
Q = [Q1, Q5, Q25, Q1000];
c = [100*rand(Float64,1)',100*rand(Float64,5)',100*rand(Float64,25)',100*rand(Float64,1000)'];
x0 = [fill(1.,1), fill(1.,5),fill(1.,25), fill(1.,1000)];

Le nécessaire étant générés, appliquons l'algorithme aux fonctions quadratiques, et vérifions si le point résultant est stationnaire en calculant le gradient.

In [7]:
res = [Array{Float64,}, fill(1.,5)', fill(1.,25)',fill(1.,1000)'];
for i in 1:4
    res[i] = GC_quad(Q[i], c[i], x0[i])
end

Fin à itération 1
Fin à itération 5
Fin à itération 25
Fin à itération 1000


In [8]:
grad = [Array{Float64,}, fill(1.,5)', fill(1.,25)', fill(1.,1000)'];
for i in 1:4
    grad[i] = res[i]'*Q[i]+c[i]
end
for i in 1:4
    print("Gradient #", i , ": \n",grad[i], "\n")
end

Gradient #1: 
[0.0]
Gradient #2: 
[1.02034e-11 2.55369e-11 1.12507e-10 3.38645e-11 1.67992e-9]
Gradient #3: 
[-0.0752351 -0.838377 0.856837 -1.6104 1.17025 0.154147 -0.583765 0.115174 0.680107 0.549322 -0.00503323 0.0192339 -2.56999 0.372975 1.32614 0.779935 0.323408 0.219634 8.65177 0.395746 -1.85479 0.859198 0.52717 0.0143842 0.00984941]
Gradient #4: 
[-378.144 60.8483 42.705 88.0383 -48.6774 -8.70151 -59.5528 78.0017 87.483 -30.8418 -65.4907 -179.915 -42.6045 -33.8849 311.117 -27.9083 15.2604 1.24293 -68.1109 -233.193 -29.6869 8.68264 -34.2542 102.03 -12.5918 -66.5438 67.6947 -19.8243 51.5197 32.2038 9.86182 -23.3751 9.93078 17.0952 -7.08019 20.1362 -87.7462 24.7051 75.0324 2.3963 54.0869 1.03211 -82.4691 54.875 -12.8662 72.962 -15.0686 -126.747 -102.724 -50.3501 -31.6461 46.4385 15.3326 -2.76824 -24.1678 -54.8928 130.277 -16.7296 -63.5163 114.257 15.4543 43.5601 27.5102 7.67728 6.51708 57.7967 70.8343 4.22987 49.5132 -27.2359 -5.19492 -17.2497 8.50004 24.5266 -28.311 13.7693 -129.5

In [9]:
gradInitial = [Array{Float64,}, fill(1.,5)', fill(1.,25)', fill(1.,1000)'];
for i in 1:4
    gradInitial[i] = x0[i]'*Q[i]+c[i]
end

for i in 1:4
    print("Norme du gradient #",i,": ",norm(grad[i]), "\n")
    print("Norme du gradient en x0 #",i,": ",norm(gradInitial[i]), "\n")
end



Norme du gradient #1: 0.0
Norme du gradient en x0 #1: 117.64108088381661
Norme du gradient #2: 1.6842502196205524e-9
Norme du gradient en x0 #2: 1229.2899594822622
Norme du gradient #3: 9.761058056457271
Norme du gradient en x0 #3: 148579.72949430838
Norme du gradient #4: 2095.4146860375963
Norme du gradient en x0 #4: 1.8528500915899563e8


### Analyse des résultats
#### Convergence
On remarque que l'algorithme n'a jamais terminé avant l'itération $n$. Ceci n'est pas surprenant, sachant que la norme du gradient de chacun des points obtenues n'est pas nulle ou inférieur à $eps() = 2.220446049250313\cdot10^{-16} $.

#### Vérification des points résultats

On peut vérifier la qualité des résultats en calculant le gradient au points résultants. Comme les points doivent être stationnaires, on s'attend à ce que les gradients soient de norme près de 0. On obtient les normes de gradient pour chacun des points résultants:

* $n = 1$: 0.0
* $n = 5$: 1.6842502196205524e-9
* $n = 25$: 9.761058056457271
* $n = 1000$: 2095.4146860375963

Pour $n = 1$ et $n = 5$, le résultat est très prometteur. On obtient une norme inférieur à $10^{-8}$. Par contre, pour $n = 25$ et $n = 1000$, on obtient comme norme des gradients aux points résultats des valeurs supérieurs à $\sqrt{n}$! ($\sqrt{n}$ est une bonne valeur de comparaison, car c'est la norme du vecteur $[1, 1, \ldots, 1]^T \in \mathbb{R}^n$)

On peut se demander toutefois si l'algorithme a tout de même été efficace, dans l'aspect de s'approcher d'un point stationnaire. Quel était la norme du gradient au point initial, soit $[1, 1, \ldots, 1]^T \in \mathbb{R}^n$. Voici les valeurs:

* $n = 1$: 117.64108088381661
* $n = 5$: 1229.2899594822622
* $n = 25$: 148579.72949430838
* $n = 1000$: 1.8528500915899563e8

On voit que l'algorithme a obtenu un point dont le gradient a grandement été réduit, d'un facteur de norme commençant à $10^{-3}$. Par contre, ça ne nous assure pas que le point obtenu est près du point minimal.

Voici une première tentative d'explication des grandes valeurs obtenues pour les gradients des points résultants de l'algorithme pour $n = 25$ et $n = 1000$. Ayant travaillé dans des dimensions moyenne/élevés, les calculs sont nombreux, et les erreurs d'approximation apparaissant dans chaque dimension se cumulent lors des opérations matricielles. Si on applique l'algorithme à nouveau et que ces « grandes » valeurs de norme de gradients persistent, cela appuierait cette hypothèse, car on n'évitera pas les erreurs d'approximations. Voyons:

In [10]:
resPrise2 = [Array{Float64,}, fill(1.,5)', fill(1.,25)',fill(1.,1000)'];
for i in 1:4
    resPrise2[i] = GC_quad(Q[i], c[i], res[i])
end
gradPrise2 = [Array{Float64,}, fill(1.,5)', fill(1.,25)', fill(1.,1000)'];
for i in 1:4
    gradPrise2[i] = resPrise2[i]'*Q[i]+c[i]
end
for i in 1:4
    print("Norme du gradient #",i,": ",norm(gradPrise2[i]), "\n")
end

Fin à itération 1
Fin à itération 5
Fin à itération 25
Fin à itération 1000
Norme du gradient #1: NaN
Norme du gradient #2: 7.944109290391274e-15
Norme du gradient #3: 0.01065746436856588
Norme du gradient #4: 1218.9117301472472


On voit que pour $n = 5$ et $n = 25$, l'algorithme a réussi à se rapprocher de plusieurs magnitude de grandeur d'un gradient nul. Pour la dimension $n = 1$, il est normal qu'on obtienne ``NAN``, car le point fourni était déjà un point stationnaire. Pour $n=1000$, toutefois, on n'arrivera pas à se rapprocher de plus d'un facteur de 2 d'un gradient nul. Par contre, il n'y a pas eu d'augmentation de la norme du gradient résultant. Ceci appuie l'hypothèse que les « grandes » valeurs de normes des gradients obtenues en dimension moyen/élevé sont potentiellement dûes à des erreurs d'approximations.

## Fonctions quadratiques et non quadratiques pour le Gr. Conj. généralisé
On souhaite maintenant tester la fonction ``GConj()``, qui est une extension de l'algorithme du gradient conjugué aux cas non-quadratiques. La différence est au niveau du calcul de $\beta_k$, qui se fait selon 4 méthodes différentes:

Soit $g_k = \nabla f(x_k)$ et $y_k = g_k - g_{k-1}$, 

* Méthode de Fletcher et Reeves (FR):
$$ \beta_k^{\text{FR}} = \frac{g_{k+1}g_{k+1}^T}{g_kg_k^T}$$

* Méthode de Polak et Ribière (PR):
$$ \beta_k^{\text{PR}} = \frac{g_{k+1}y_{k+1}^T}{g_kg_k^T}$$

* Méthode de Hestenes et Stiefel (HS):
$$ \beta_k^{\text{HS}} = \frac{g_{k+1}y_{k+1}^T}{y_{k+1}d_k}$$

* Méthode de Hager et Zhang (HZ):
$$ \beta_k^{\text{HZ}} = \frac{1}{y_{k+1}d_k}\left(y_{k+1} - 2d_k^T \frac{||y_{k+1}||^2}{y_{k+1}d_k} \right)g_{k+1}^T$$

Nécessairement, il faudra tester les fonctions avec les différentes variantes du calcul de $\beta_k$.

### Fonctions de tests
Selon l'énoncé du devoir, on veut tester l'algorithme avec 2 fonctions quadratiques et 2 fonctions non-quadratiques.
Il sera repris les fonctions quadratiques de dimension 5 et 25 utilisées précédemment, afin de pouvoir comparer les résultats de convergence. De plus, dans les fonctions non-quadratiques, il serait souhaitable d'avoir une fonction similaire à une fonction quadratique, disons polynomiale. Celle-ci sera basée sur la fonction de dimension 5, à laquelle on ajoutera des termes non-quadratiques, soit de degré 3. La deuxième fonction non-quadratique sera une combinaison de fonctions trigonométriques, quadratique et exponentielle. Les voici:

In [11]:
# Fonctions quadratiques
f1(x::Vector) = x'*Q[2]*x/2 + c[2]*x # Dimension 5
f2(x::Vector) = x'*Q[3]*x/2 + c[3]*x # Dimension 25

# Fonctions non-quadratiques
f3(x::Vector) = x'*Q[2]*x/2 - c[2]*x + x[1]^4 + x[2]^4 + x[3]^4 + x[4]^4 #Dimension 5
f4(x::Vector) = -exp(1 - (x[1]-1.9)^2 - (x[2]-1.7)^2 - (x[3]-2.5)^2) + 0.5*x[1]^2 + 10*cos(x[1]) #Dimention 3

f4 (generic function with 1 method)

Nous sommes assuré que les formes quadratiques sont définies positives, par leur construction.

Pour les fonctions non-quadratiques, on remarque $f_3$ est la forme $f2$ ajouté de $x_1^4 + x_2^4 + x_3^4 + x_4^4$, et dont la composante linéaire $cx$ est négative plutôt que positive. En ajoutant un polynôme d'ordre 4, je suis assuré qu'un minimum existe.

La fonction $f_4 = -e^{1-(x_1-1.9)^2 - (x_2-1.7)^2 - (x_3-2.5)^2} + \frac{x_1^2}{2} + 10\cos{x_1}$ est une fonction tiré d'un exercice du *Stewart, J. Calcul à plusieurs variables, 2ème édition*, page 229, #34. On voit que la partie exponentielle est convexe, que la partie polynomiale aussi et que la partie trigonométrique n'affectera pas la présence d'un minimum global, car borné, malgré qu'il puisse cassé la convexité en certains points.

### Méthodologie
Nous procéderons au test des fonctions en partant du point $[1, 1, \ldots, 1]^T \in \mathbb{R}$, le même point de départ utilisé pour tester l'algorithme limité aux fonctions quadratiques.

Les critère d'arrêt, soit $\epsilon$ et la limite d'itérations seront posés à $\sqrt{\epsilon_{\text{machine}}}$ et à 10000. Ainsi, on cherche une bonne précision (un gradient très près de 0) et on empêche l'algorithme de rouler trop longtemps.

Ce test sera fait selon les 4 variantes du calcul de $\beta_k$, appliqués à chacune des fonctions. Les résultats seront analysés et discutés ensuite.

Les $\tau$ seront choisis selon une légère grille pour voir si le choix affecte la convergence, choisis parmies ``grille_τ = [0.05, 0.2, 0.4]`` 

In [44]:
f = [f1, f2, f3, f4]
grille_τ = [0.05, 0.2, 0.4]
resTest2 = [];
gradTest2 = [Array{Float64,}, fill(1.,25)', fill(1.,5)',fill(1.,3)'];
x_depart = [fill(1.,5)', fill(1.,25)', fill(0.,5)',fill(1.,3)'];

for τ in grille_τ
    print("Pour τ = ", τ, "\n")
    for i in 1:4
        print(".Fonction ", i, "\n")
        for β in (FR, PR, HS, HZ)
            print("..Variante ", β, "\n")
            resTest2 = push!(resTest2, GConj(f[i], x_depart[i]', τ, β, 10000))
        end
    end
end

Pour τ = 0.05
.Fonction 1
..Variante FR
...Fin à itération 10000
..Variante PR
...Fin à itération 10003
..Variante HS
...Fin à itération 10000
..Variante HZ
...Fin à itération 10000
.Fonction 2
..Variante FR
...Fin à itération 10000
..Variante PR
...Fin à itération 10015
..Variante HS
...Fin à itération 10000
..Variante HZ
...Fin à itération 10000
.Fonction 3
..Variante FR
...Fin à itération 10000
..Variante PR
...Fin à itération 10000
..Variante HS
...Fin à itération 10000
..Variante HZ
...Fin à itération 10000
.Fonction 4
..Variante FR
...Fin à itération 10001
..Variante PR
...Fin à itération 10002
..Variante HS
...Fin à itération 10000
..Variante HZ
...Fin à itération 10000
Pour τ = 0.2
.Fonction 1
..Variante FR
...Fin à itération 10000
..Variante PR
...Fin à itération 10003
..Variante HS
...Fin à itération 10000
..Variante HZ
...Fin à itération 10000
.Fonction 2
..Variante FR
...Fin à itération 10000
..Variante PR
...Fin à itération 10000
..Variante HS
...Fin à itération 10000
..Va

In [45]:
∇f = []
for τ in grille_τ
    print("Pour τ = ", τ, "\n")
    for j in 1:4
        print(".Fonction ", j, "\n")
        ∇f = push!(∇f,x -> ForwardDiff.gradient(f[j], x))
        for i in 1:4
            print(".. Variante ", [FR, PR, HS, HZ][i], "\n")
            print("   ",norm(∇f[j](resTest2[4*(j-1)+i])), "\n")
        end
    end
end

Pour τ = 0.05
.Fonction 1
.. Variante FR
   1.0470517282669527e-6
.. Variante PR
   2.3010960495389834e-6
.. Variante HS
   2.6986694229629273e-6
.. Variante HZ
   2.20273111971459e-6
.Fonction 2
.. Variante FR
   1.3573082113262508e-5
.. Variante PR
   3.563360340161229e-5
.. Variante HS
   2.3868210193965534e-5
.. Variante HZ
   9.54280384256263e-6
.Fonction 3
.. Variante FR
   3.294571105767143e-6
.. Variante PR
   3.7216081185116453e-6
.. Variante HS
   2.6314419138128664e-6
.. Variante HZ
   2.1090351075133664e-6
.Fonction 4
.. Variante FR
   4.90987601757039e-8
.. Variante PR
   1.6818596293433075e-8
.. Variante HS
   1.982807074085713e-8
.. Variante HZ
   2.9692985847873388e-8
Pour τ = 0.2
.Fonction 1
.. Variante FR
   1.0470517282669527e-6
.. Variante PR
   2.3010960495389834e-6
.. Variante HS
   2.6986694229629273e-6
.. Variante HZ
   2.20273111971459e-6
.Fonction 2
.. Variante FR
   1.3573082113262508e-5
.. Variante PR
   3.563360340161229e-5
.. Variante HS
   2.3868210193965

### Analyse des résultats
#### Convergence de l'algorithme
Dans tous les cas, autant pour les fonctions quadratiques que non-quadratiques, l'algorithme a effectué $\pm$10000 itérations, soit la limite posée initialement. De plus, dans tous les cas, l'algorithme a renvoyé un point dont le gradient est très près de 0, soit des gradients de norme au plus $10^{-4}$. Je conclus que l'algorithme fonctionne et converge vers un minimum local si celui-ci est localement existant et accessible.

#### Choix de $\tau$
On remarque que le choix de $\tau$ avec un critère d'itération max à 10000 n'affecte ni le temps de convergence, ni la norme du gradient résultant. Il se peut bien sûr que si on pose la limite d'itération plus basse, le choix de $\tau$ affecte les déplacements fait. Pour la comparaison des résultats, nous prendrons ceux obtenus avec $\tau = 0.2$

#### Choix de variante de calcul de $\beta_k$
Dans tous les cas, les variantes de calcul de $\beta_k$ résultent en une différence d'une magnitude d'au plus 10 entre les normes de gradients des points obtenus. Prenons par exemple le résultat de l'algorithme sur $f_2$, avec $\tau= 2$:
* Variante FR:
   1.3573082113262508e-5
* Variante PR:
   3.563360340161229e-5
* Variante HS:
   2.3868210193965534e-5
* Variante HZ:
   9.54280384256263e-6

Dans ce cas-ci, le ratio entre la plus grande et la plus petite norme du gradient au point résultant est d'environ 3.73. De plus, la variante de Hager et Zhang a donné le meilleur résultat. 

Pour la fonction $f_1$, la méthode de Fletcher-Reeves l'emporte, pour la fonction $f_3$, celle d'Hager et Zhang est plus fructueuse et pour $f_4$, c'est la méthode de Polak et Ribière qui donne la norme au point résultant le plus petit. 

#### Comparaison GC version quadratique versus GC version étendue
Les fonctions $f1$ et $f2$ ont été reprises des test de l'algorithme du Gradient Conjugué appliqué seulement aux fonctions quadratiques. Ainsi, on peut comparer les résultats obtenus par les deux algorithmes. Voyons les résultats:

Commençons avec la norme du gradient du point stationnaire trouvé pour la fonction:
##### $f_1$
* Algo ``GC_quad``: 1.6842502196205524e-9
* Algo ``GConj`` avec:
    * Variante FR: 1.0470517282669527e-6
    * Variante PR: 2.3010960495389834e-6
    * Variante HS: 2.6986694229629273e-6
    * Variante HZ: 2.20273111971459e-6
    
##### $f_2$
* Algo ``GC_quad``: 9.761058056457271
* Algo ``GConj`` avec:
    * Variante FR: 1.3573082113262508e-5
    * Variante PR: 3.563360340161229e-5
    * Variante HS: 2.3868210193965534e-5
    * Variante HZ: 9.54280384256263e-6

On observe deux phénomènes. Premièrement, l'algorithme ``GC_quad`` a obtenu de meilleurs résulats au niveau de la fonction de dimension 5. Deuxièmement, c'est l'algorithme ``GConj`` qui remporte le titre d'algorithme le plus efficace au niveau de la fonction quadratique de dimension 25.

La théorie suggère que dans les deux cas, les résultats sont supposés être exactement les mêmes, puisque les variantes du calcul de $\beta_k$ par FR, PR, HS et HZ reviennent au même calcul de $\beta_k$ dans le cas de l'algorithme pour les fonctions quadratiques et puisque l'algorithme est supposé convergé en au plus $n$ itérations, où $n$ est la dimension du domaine de la fonction.

Hors, la théorie ne prédit pas les erreurs de calculs de la machine. On a vu à l'analyse des résultats du premier algorithme que l'augmentation de dimension du domaine de la fonction semble causer une augmentation de la norme du gradient de la fonction au point obtenu par l'algorithme, ce que j'ai expliqué par des erreurs d'approximation de la machine. La force de l'algorithme du gradient conjugué étendu, c'est qu'il ne s'arrête pas à $n$ itérations, de base, et continuera de rouler tant qu'il ne converge pas vers une plus petite valeur de norme de gradient, où qu'il frappe la limite d'itération selon le critère d'arrêt. 

Cette hypothèse n'explique pas pourquoi l'algo ``GConj`` ne donne pas un meilleur résulat pour la fonction $f_1$. J'expliquerais ceci par la façon dont est calculée la dérivée de la fonction. En effet, dans l'algorithme ``GC_quad``, on utilise la dérivée théorique, soit: $ \nabla f(x) = x^TQ + c $, directement calculée par la matrice $Q$ et le vecteur $c$. Hors, dans l'algorithme ``GConj``, la dérivée est calculée par la ligne 

``g = x -> ForwardDiff.gradient(f, x)``.
Je considère cette fonction comme une boîte noire au niveau de la précision du résultat, et j'expliquerais donc l'écart de précision par le manque de précision plausible de la fonction ``ForwardDiff.gradient``.

